In [9]:
import os

import numpy as np
import pandas as pd

import json

In [10]:
def process_action(action):
    action = action.split('\n')
    content = {}
    content['alternatives'] = int(action[1].split('=')[1].strip())
    depths = [int(d) for d in action[2].split('=')[1].strip().split(' ')]
    content['mean_depth'] = float(np.mean(depths))
    content['mean_depth_std'] = float(np.std(depths))
    content['max_depth'] = int(action[3].split(',')[2].split('=')[1].strip())
    content['score'] = int(action[3].split(',')[1].split('=')[1].strip())
    return content

In [11]:
def process_log(log, h1, h2):
    content = {}
    log = log.split('\n\n')
    content['W'] = int(log[0].split(',')[0].split('=')[1].strip())
    content['B'] = int(log[0].split(',')[1].split('=')[1].strip())
    content['reduction'] = int(log[-1].split('\n')[0].split(',')[0].split('=')[1].strip())
    content['alpha'] = int(log[-1].split('\n')[0].split(',')[1].split('=')[1].strip())
    content['beta'] = int(log[-1].split('\n')[0].split(',')[2].split('=')[1].strip())
    content['time'] = float(log[-1].split('\n')[1].split(',')[0].split('=')[1].strip())
    content['W_time'] = float(log[-1].split('\n')[1].split(',')[1].split('=')[1].strip())
    content['B_time'] = float(log[-1].split('\n')[1].split(',')[2].split('=')[1].strip())
    content['winner'] = int(h1) if log[-2].split('=')[1].strip() == 'MAX' else int(h2)
    log = log[1:-3]
    content['moves'] = len(log)

    for i, action in enumerate(log):
        a = process_action(action)
        if i%2 == 0:
            t = 'W'
        else:
            t = 'B'
        if i==0 or i==1:
            content[f'{t}_alter'] = []
            content[f'{t}_mean_depth'] = []
            content[f'{t}_max_depth'] = []
            content[f'{t}_score'] = []
            
        content[f'{t}_alter'].append(a['alternatives'])
        content[f'{t}_mean_depth'].append(a['mean_depth'])
        content[f'{t}_max_depth'].append(a['max_depth'])
        content[f'{t}_score'].append(a['score'])
        
    # print(content)
            
    for t in ['W', 'B']:
        for s in ['alter', 'mean_depth', 'max_depth', 'score']:
            content[f'{t}_{s}'] = f'{round(float(np.mean(content[f'{t}_{s}'])))} +/- {round(float(np.std(content[f'{t}_{s}'])))}'
    return content

In [12]:
matchups = []
for fi in os.listdir('logs'):
    with open(f"logs/{fi}") as f:
        log = f.read()
        h1, h2 = fi.split('_')
        content = process_log(log, h1, h2)
        matchups.append(content)

In [13]:
df = pd.DataFrame(matchups)

In [14]:
df.sort_values(by='W')

,W,B,reduction,alpha,beta,time,W_time,B_time,winner,moves,W_alter,W_mean_depth,W_max_depth,W_score,B_alter,B_mean_depth,B_max_depth,B_score
6,1,5,160790,68369,92421,10.0,4.6,5.5,5,94,18 +/- 12,9 +/- 3,12 +/- 4,-9 +/- 11,26 +/- 6,8 +/- 3,11 +/- 4,-16 +/- 16
4,1,4,1934148,1064025,870123,99.8,46.8,53.0,1,137,28 +/- 7,9 +/- 6,10 +/- 8,28 +/- 18,13 +/- 13,9 +/- 6,10 +/- 7,23 +/- 18
10,1,3,863149,482956,380193,101.3,28.9,72.4,3,84,31 +/- 14,14 +/- 4,18 +/- 5,-13 +/- 22,28 +/- 10,14 +/- 5,18 +/- 5,-22 +/- 26
1,3,1,245305,167748,77557,32.1,26.3,5.9,3,171,26 +/- 10,8 +/- 4,10 +/- 6,17 +/- 20,14 +/- 11,8 +/- 3,10 +/- 6,14 +/- 21
3,3,5,369044,149396,219648,55.3,32.4,22.9,5,218,18 +/- 13,8 +/- 3,11 +/- 5,-2 +/- 14,26 +/- 7,8 +/- 3,11 +/- 4,-4 +/- 14
2,3,4,587571,240738,346833,74.7,57.9,16.8,4,86,25 +/- 10,11 +/- 5,16 +/- 7,-9 +/- 8,26 +/- 13,12 +/- 5,16 +/- 6,-14 +/- 15
7,4,5,1310922,795912,515010,67.0,38.3,28.7,5,184,12 +/- 13,7 +/- 5,9 +/- 7,-17 +/- 21,27 +/- 9,7 +/- 5,9 +/- 7,-18 +/- 20
0,4,3,583471,261782,321689,54.0,16.4,37.5,3,114,17 +/- 17,9 +/- 5,12 +/- 7,-15 +/- 9,24 +/- 8,9 +/- 5,12 +/- 7,-21 +/- 17
11,4,1,1321834,818082,503752,65.6,38.6,26.9,4,115,25 +/- 10,10 +/- 5,12 +/- 8,11 +/- 18,17 +/- 11,10 +/- 5,13 +/- 7,9 +/- 14
5,5,4,1926657,1058299,868358,105.0,49.0,55.9,5,61,34 +/- 6,13 +/- 6,16 +/- 7,16 +/- 19,25 +/- 12,13 +/- 6,17 +/- 7,9 +/- 10


In [15]:
df.to_csv('data/benchmark.csv', index=False)